In [1]:
# All modules that  are needed.
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline   
import rubin_sim.maf as maf
from rubin_sim.maf.maps import dustMap





from PulsatingStarRecovery_ANGELO import PulsatingStarRecovery


In [2]:
from rubin_sim.data import get_baseline

opsim_fname='/sims_maf/fbs_2.0/baseline/baseline_v2.0_10yrs.db'

#runName='baseline_v2.0_10yrs.db'

runName = os.path.split(opsim_fname)[-1].replace('.db', '')
print(runName)

# Connect to the database so we can use it with MAF.
opsim_db = maf.OpsimDatabase(opsim_fname)
#Finding available database columns with which to express constraints, slices, and metrics
#opsim_db.get_column_names("observations")



baseline_v2.0_10yrs


In [17]:
# LMC 
slicer = maf.UserPointsSlicer(
       80.89416666666666,  -69.75611111111111, latLonDeg=True)#,latLonDeg= #badval=-666
    #)
    # Add any additional information about each object to the slicer
#Distance in Mpc
slicer.slicePoints["distance"] = 0.05

print(slicer.slicePoints)

maps = ['DustMap']

{'sid': array([0]), 'ra': array([1.41186955]), 'dec': array([-1.21747381]), 'distance': 0.05}


In [18]:
#This cell  build DataFrame df that contains  the  list of blending stars with magnitudes . Can be empty if you don't wont to consider blending or if you are offline.
#In that case comment from import to print

dati=[]
df=pd.DataFrame(dati,columns=[])

#import dl.queryClient as qc
#test_ra = test_ra
#test_dec = test_dec
#table = 'lsst_sim.simdr2'
#radius = 0.0002
#query = "select ra,dec,umag,gmag, rmag, imag, zmag, ymag, mu0, gc, mass from {} where q3c_radial_query(ra, dec, {}, {}, {})".format(table,test_ra,test_dec,radius)
#df = qc.query(sql=query ,timeout=1000, fmt='pandas') 
#print(df)

 

In [19]:
#parameters that must be inserted to run the metric PulsatingStarRecovery.py.Possible filenames are (https://github.com/MARCELLADC/PulsatingStarRecovery):
filename='./RRab.csv'
#filename='./RRc.csv'   #ascii file with the template of pulsating star

sigmaFORnoise=1   # sigma for the computation of the error on the magnitude
do_remove_saturated=True    #Remove from temporal series saurated visits  
factorForDimensionGap=0.75  #parameter used in LcSampling 




#mymetric = PulsatingStarRecovery(filename,sigmaFORnoise,do_remove_saturated,numberOfHarmonics,factorForDimensionGap,df)
#if you are interested in other metrics
#maf.BaseMetric.help(doc=True)

In [20]:
##### Define a sqlconstraint, if we need to just use a (large) subset of the opsim visits
#sqlconstraint = None   # no constraint, make all visits available

# Examples of other potentially useful sqlconstraints:
# sqlconstraint = 'filter = "r"'  # just select the visits in a particular filter
# sqlconstraint = 'note not like "%DD%"'  # don't choose any of the DD field visits
# sqlconstraint = 'night < 700'  # only use visits in the first year of the survey
#sqlconstraint = 'night < 365' 
#sqlconstraint = 'proposalID !=1 and note not like "%DD%"'
#sqlconstraint = 'night < 3650 and proposalID !=1 and note not like "%DD%"' 
sqlconstraint = 'night < 3650'

In [ ]:
numberOfHarmonics=3
mymetric = PulsatingStarRecovery(filename,sigmaFORnoise,do_remove_saturated,numberOfHarmonics,factorForDimensionGap,df)
bundle= maf.MetricBundle(mymetric, slicer, sqlconstraint,maps)
g = maf.MetricBundleGroup({'PulsatingStarRecovery':bundle }, opsim_db, outDir='./', resultsDb=None)
g.runAll()

./RRab.csv
Querying database observations with constraint night < 3650 for columns ['rotSkyPos', 'numExposures', 'observationStartMJD', 'fieldDec', 'visitExposureTime', 'seeingFwhmEff', 'airmass', 'filter', 'skyBrightness', 'night', 'fiveSigmaDepth', 'fieldRA']
Found 2085013 visits
Running:  ['PulsatingStarRecovery']
 distance=, ebv=, e dmod=
0.05
0.6333438158035278
18.494850021680094
Useful (at all S/N and NOT saturated)  Nvisits in ugrizy bands
57 73 197 199 163 171
1.2093260357638833
23.100267274489898
21.39691148757834
22.588962532222972
20.234718507554998
19.943883036622275
19.77368669093968


/data0/sw.tmpfs/lsst_stack-2021.10.13/anaconda3/lib/python3.8/site-packages/gatspy/periodic/lomb_scargle_fast.py:445: UserWarning: The score() method defaults to a slower O[N^2] algorithm.
Use the score_frequency_grid() method to access the fast FFT-based algorithm.
To silence this warning, set ``silence_warnings=True``
  warnings.warn("The score() method defaults to a slower O[N^2] "


Noise level (median vs mean)
0.00883424805146138 0.012976457560317105
Finding optimal frequency:
 - Estimated peak width = 0.0018
 - Using 5 steps per peak; omega_step = 0.000359
 - User-specified period range:  0.42 to 0.79
 - Computing periods at 19055 steps
Zooming-in on 5 candidate peaks:
 - Computing periods at 995 steps


In [26]:
import pickle
with open('RRab.p','wb') as fout:
       pickle.dump(bundle.metricValues.data, fout)  

In [33]:
import pickle
with open('RRab.p','rb') as fout:
    result_RRab=pickle.load(fout)
